# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

cat: publications.tsv: No such file or directory


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

/var/folders/n8/kg9rwbrs1wd0c6nsb4tn2hvc0000gn/T/ipykernel_6693/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
publications = pd.read_csv("publications.csv", sep="|", skipinitialspace=True, header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2024-04-20,5G-WAVE: A Core Network Framework with Decentr...,IEEE International Conference on Computer Comm...,NaN,"Pragya Sharma, Tolga Atalay, Hans-Andrew Gibbs...",5g-wave,https://arxiv.org/abs/2404.13242
1,2020-03-09,Adaptive Flow-Level Scheduling for the IoT MAC,2020 International Conference on COMmunication...,NaN,"Pragya Sharma, Jayakrishnan Nair, Raman Singh,...",adaptive-mac-for-iot,https://doi.org/10.1109/COMSNETS48256.2020.902...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2020-03-09-adaptive-mac-for-iot.md 2024-04-20-5g-wave .md
2020-03-09-paper-1.md              2024-04-20-5g-wave.md


In [17]:
!cat ../_publications/2024-04-20-5g-wave.md

---
title: "5G-WAVE: A Core Network Framework with Decentralized Authorization for Network Slices"
collection: publications
permalink: /publication/2024-04-20-5g-wave
date: 2024-04-20
venue: 'IEEE International Conference on Computer Communications (INFOCOM) 2024 '
paperurl: 'https://arxiv.org/abs/2404.13242'
citation: 'Pragya Sharma, Tolga Atalay, Hans-Andrew Gibbs, Dragoslav Stojadinovic, Angelos Stavrou, Haining Wang '
---
[Download paper here](https://arxiv.org/abs/2404.13242)

Recommended citation: Pragya Sharma, Tolga Atalay, Hans-Andrew Gibbs, Dragoslav Stojadinovic, Angelos Stavrou, Haining Wang 